# Import python packages

In [1]:
import os
import cv2 as cv2
import numpy as np
import math
import sys

from scipy import ndimage
from scipy import ndimage as ndi 
from skimage.transform import rotate, hough_circle, hough_circle_peaks, rescale, resize
from skimage import util
from skimage import draw
from skimage import io, color, data, measure, filters, exposure, img_as_ubyte, img_as_float
from skimage.measure import regionprops, label
from skimage.morphology import binary_dilation, disk, binary_erosion, convex_hull_image, remove_small_objects, erosion, remove_small_holes, dilation
from skimage.segmentation import morphological_geodesic_active_contour, inverse_gaussian_gradient, checkerboard_level_set, morphological_chan_vese
from skimage.exposure import match_histograms
from skimage.segmentation import clear_border, active_contour
from skimage.feature import canny
from skimage.filters import gaussian, unsharp_mask, threshold_local, threshold_otsu
from skimage.transform import AffineTransform, warp

from PIL import ImageFilter

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd


# Functions

In [ ]:
def get_the_black_part(img):
    
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV) 
    low_hsv = np.array([0,0,0],dtype = np.uint8)
    upper_hsv = np.array([180,255,46],dtype = np.uint8)
    mask = cv2.inRange(hsv,low_hsv,upper_hsv)
    
    return mask

def get_qr_points(img):
    
    qrCodeDetector = cv2.wechat_qrcode_WeChatQRCode("QRcode_backbone\\detect.prototxt", 
                                                    "QRcode_backbone\\detect.caffemodel", 
                                                    "QRcode_backbone\\sr.prototxt", 
                                                    "QRcode_backbone\\sr.caffemodel")
    qr_info, points = qrCodeDetector.detectAndDecode(img)
    
    return points, qr_info

def get_qr_crop_mask_V2(img, points):

    for i in points:
        coord1 = i[0]
        coord2 = i[1]
        coord3 = i[2]
        coord4 = i[3]
        point1 = (int(coord1[0]), int(coord1[1]))
        point2 = (int(coord2[0]), int(coord2[1]))
        point3 = (int(coord3[0]), int(coord3[1]))
        point4 = (int(coord4[0]), int(coord4[1]))
    zero_img = np.zeros(img.shape[:2], np.uint8)
    cv2.line(zero_img, point1, point2, (255,255,255), 10)
    cv2.line(zero_img, point1, point4, (255,255,255), 10)
    cv2.line(zero_img, point3, point2, (255,255,255), 10)
    cv2.line(zero_img, point3, point4, (255,255,255), 10)
    qr_hull = convex_hull_image(zero_img).astype('uint8')
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15, 15))
    qr_mask = cv2.dilate(qr_hull, kernel)

    return qr_mask

def display_object_from_original(img, label_img):

    mask = label_img > 0

    r = img[:, :, 0] * mask
    g = img[:, :, 1] * mask
    b = img[:, :, 2] * mask

    img_temp = np.dstack([r, g, b])
    
    return img_temp

def function_good_match(des1,des2, delta = 0.5):
    bfm = cv2.BFMatcher()
    matches = bfm.knnMatch(des1, des2, k=2)
    good_match = []
    for m1, m2 in matches:
        if m1.distance < delta * m2.distance:
            good_match.append(m1)
    return good_match

def angle_between(p1, p2, p3):
    # calcaulate the angle 
    v1 = p1 - p2
    v2 = p3 - p2
    angle = np.math.atan2(np.linalg.det([v1,v2]),np.dot(v1,v2))
    return np.degrees(angle)

def is_rectangle(contour, angle_threshold=10, ratio_threshold=0.2):
    # Contour approximation
    epsilon = 0.05 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    if len(approx) == 4:
        # check the angle
        for i in range(4):
            angle = abs(angle_between(approx[i][0], approx[(i+1)%4][0], approx[(i+2)%4][0]))
            if abs(angle - 90) > angle_threshold:
                return False

        # Check the ratio of side lengths
        edge_lengths = [np.linalg.norm(approx[i][0] - approx[(i+1)%4][0]) for i in range(4)]
        if max(edge_lengths) / min(edge_lengths) > 1 + ratio_threshold:
            return False

        return True

    return False

In [ ]:
image_list = os.listdir(img_input_folder)    ### 'img_input_folder' refers to the path of the folder to be analyzed; you need to specify it first.

if os.path.exists(img_output_folder):   ### 'img_output_folder' refers to the path where the output is saved; you need to specify it first. 
    pass
else:
    os.makedirs(img_output_folder)

for j in range(len(image_list)):

    image_name = image_list[j].split('.')[0]

    image_path = os.path.join(img_input_folder, image_list[j])

    print("*********processing image %s***********" % (image_list[j]))

    output_image_path = os.path.join(img_output_folder, image_name + '.png')

    base_image_path = os.path.join(img_input_folder, image_list[0])   ### define the baseline image

    base_image = cv2.imread(base_image_path)

    base_qr_points, base_qr_info = get_qr_points(base_image)
    if base_qr_points != ():
    
        get_base_qr_mask = get_qr_crop_mask_V2(base_image, base_qr_points)
    
    else:
        base_image_path = os.path.join(img_input_folder, image_list[1])

        base_image = cv2.imread(base_image_path)
        base_qr_points, base_qr_info = get_qr_points(base_image)
        get_base_qr_mask = get_qr_crop_mask_V2(base_image, base_qr_points)

    base_qr_part = display_object_from_original(base_image, get_base_qr_mask)
    
    input_image = cv2.imread(image_path)

    qr_points, qr_info = get_qr_points(input_image)

    if qr_points!=():
        
        dst_PT_img = input_image

        qr_mask = get_qr_crop_mask_V2(input_image, qr_points)

        edge_img = np.logical_xor(qr_mask, binary_erosion(qr_mask, disk(2)))
        edge = edge_img.astype(np.uint8)
        contours, _ = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if is_rectangle(cnt)==True:
                # Draw the contour
                qr_mask = cv2.dilate(qr_mask, disk(2))
                qr_part = display_object_from_original(input_image, qr_mask)
                img_gray1 = cv2.cvtColor(base_qr_part, cv2.COLOR_BGR2GRAY)
                img_gray2 = cv2.cvtColor(qr_part, cv2.COLOR_BGR2GRAY)

                sift= cv2.SIFT_create()

                sift_img1 = img_as_ubyte(img_gray1)
                sift_img2 = img_as_ubyte(img_gray2)

                # Extraction of SIFT feature points and feature descriptions
                kp1, des1 = sift.detectAndCompute(sift_img1,None)
                kp2, des2 = sift.detectAndCompute(sift_img2,None)

                goodMatch = function_good_match(des1,des2)

                # Use the KNN algorithm to find the two nearest data points.
                # If the ratio of the closest value to the second-closest value is greater than a predefined value,
                # Keep this closest value and consider it and the point it matches as a good match.
                MIN_MATCH_COUNT = 10

                if len(goodMatch)>MIN_MATCH_COUNT:
                    src_pts = np.float32([kp1[m.queryIdx].pt for m in goodMatch]).reshape(-1,1,2)
                    dst_pts = np.float32([kp2[m.trainIdx].pt for m in goodMatch]).reshape(-1,1,2)

                    M_f, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
                    matchesMask = mask.ravel().tolist()

                    h,w = sift_img1.shape
                    pts_f = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)

                    dst = cv2.perspectiveTransform(pts_f, M_f)
                    sift_img2 = cv2.polylines(sift_img2, [np.int32(dst)], True, 255,3, cv2.LINE_AA)
                else:
                    matchesMask = None
                                            
                dst_PT_img = cv2.warpPerspective(input_image, M_f,(qr_part.shape[1],qr_part.shape[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        
                cv2.imwrite(output_image_path, dst_PT_img)
            else:
                
                continue
        
    else:

        print('can not detect qrcode')
        continue